# Quantum Game of Life: Practical Examples & Exercises

This notebook contains hands-on examples and exercises to help you master the QGoL Hamiltonian implementation.

**Prerequisites**: Complete the main tutorial notebook first!

---

In [ ]:
# Setup
import pennylane as qml
from pennylane import numpy as np
import matplotlib.pyplot as plt
from itertools import product, combinations
import warnings
warnings.filterwarnings('ignore')

# Import the QGoL class from the main implementation
# (In practice, you'd import from quantum_game_of_life.py)

class QuantumGameOfLife:
    """Quantum Game of Life implementation."""
    
    def __init__(self, grid_size=(2, 2), periodic=True):
        self.rows, self.cols = grid_size
        self.n_qubits = self.rows * self.cols
        self.periodic = periodic
        self.dev = qml.device('default.qubit', wires=self.n_qubits)
        
    def coord_to_qubit(self, row, col):
        return row * self.cols + col
    
    def qubit_to_coord(self, qubit):
        return qubit // self.cols, qubit % self.cols
        
    def get_neighbors(self, row, col):
        neighbors = []
        for dr, dc in [(-1,-1), (-1,0), (-1,1), (0,-1), (0,1), (1,-1), (1,0), (1,1)]:
            if self.periodic:
                new_row = (row + dr) % self.rows
                new_col = (col + dc) % self.cols
            else:
                new_row = row + dr
                new_col = col + dc
                if new_row < 0 or new_row >= self.rows or new_col < 0 or new_col >= self.cols:
                    continue
            neighbors.append(self.coord_to_qubit(new_row, new_col))
        return neighbors
    
    def build_neighbor_projector(self, site, k):
        row, col = self.qubit_to_coord(site)
        neighbors = self.get_neighbors(row, col)
        n_neighbors = len(neighbors)
        
        if k > n_neighbors:
            return [], []
        
        coeffs = []
        obs = []
        
        for alive_neighbors in combinations(neighbors, k):
            alive_set = set(alive_neighbors)
            coeff_product = (0.5) ** n_neighbors
            
            for z_pattern in product([0, 1], repeat=n_neighbors):
                pauli_ops = []
                term_coeff = coeff_product
                
                for idx, neighbor in enumerate(neighbors):
                    if neighbor in alive_set:
                        if z_pattern[idx] == 1:
                            pauli_ops.append(qml.PauliZ(neighbor))
                            term_coeff *= -1
                    else:
                        if z_pattern[idx] == 1:
                            pauli_ops.append(qml.PauliZ(neighbor))
                
                if len(pauli_ops) == 0:
                    obs_term = qml.Identity(site)
                else:
                    obs_term = pauli_ops[0]
                    for op in pauli_ops[1:]:
                        obs_term = obs_term @ op
                
                coeffs.append(term_coeff)
                obs.append(obs_term)
        
        return coeffs, obs
    
    def build_hamiltonian(self):
        all_coeffs = []
        all_obs = []
        
        for site in range(self.n_qubits):
            for k in [2, 3]:
                proj_coeffs, proj_obs = self.build_neighbor_projector(site, k)
                for c, obs in zip(proj_coeffs, proj_obs):
                    full_obs = qml.PauliX(site) @ obs
                    all_coeffs.append(c)
                    all_obs.append(full_obs)
        
        return qml.Hamiltonian(all_coeffs, all_obs)
    
    def evolve(self, H, time, n_steps, initial_state=None):
        @qml.qnode(self.dev)
        def circuit():
            if initial_state is not None:
                qml.QubitStateVector(initial_state, wires=range(self.n_qubits))
            qml.TrotterProduct(H, time, n=n_steps)
            return qml.probs(wires=range(self.n_qubits))
        return circuit()

print("✓ Setup complete!")

---

## Example 1: Exploring Classical GoL Patterns

Let's encode some famous Game of Life patterns and see how they evolve in the quantum version!

In [ ]:
def create_pattern_state(pattern, grid_size):
    """
    Create a quantum state from a classical GoL pattern.
    
    Args:
        pattern: 2D array (0=dead, 1=alive)
        grid_size: tuple (rows, cols)
    
    Returns:
        state vector
    """
    rows, cols = grid_size
    n_qubits = rows * cols
    
    # Convert pattern to binary index
    flat_pattern = pattern.flatten()
    binary_str = ''.join(str(int(x)) for x in flat_pattern)
    state_index = int(binary_str, 2)
    
    # Create state vector
    state = np.zeros(2**n_qubits)
    state[state_index] = 1.0
    
    return state

# Define some classic patterns
patterns = {
    'Block (Still Life)': np.array([
        [1, 1],
        [1, 1]
    ]),
    'Blinker (Oscillator)': np.array([
        [0, 1, 0],
        [0, 1, 0],
        [0, 1, 0]
    ]),
    'Checkerboard': np.array([
        [1, 0],
        [0, 1]
    ]),
}

print("Classic Game of Life Patterns:\n")
for name, pattern in patterns.items():
    print(f"{name}:")
    print(pattern)
    print()

### Simulate the Block Pattern

In [ ]:
# Create 2×2 grid
qgol = QuantumGameOfLife(grid_size=(2, 2), periodic=True)
H = qgol.build_hamiltonian()

# Create block pattern state
block_state = create_pattern_state(patterns['Block (Still Life)'], (2, 2))

# Simulate evolution
times = np.linspace(0, 3, 30)
fidelities = []

for t in times:
    probs = qgol.evolve(H, t, 20, block_state)
    # Fidelity = probability of staying in initial state
    fidelity = probs[15]  # |1111⟩ is index 15
    fidelities.append(fidelity)

# Plot
plt.figure(figsize=(10, 4))
plt.plot(times, fidelities, 'b-', linewidth=2)
plt.xlabel('Time', fontsize=12)
plt.ylabel('Fidelity to Initial State', fontsize=12)
plt.title('Block Pattern Stability Under QGoL Evolution', fontsize=13)
plt.grid(True, alpha=0.3)
plt.ylim([0, 1.1])
plt.show()

print(f"Initial fidelity: {fidelities[0]:.4f}")
print(f"Final fidelity (t={times[-1]:.1f}): {fidelities[-1]:.4f}")
print("\nNote: In classical GoL, a block is a 'still life' - it never changes.")
print("In QGoL, quantum fluctuations cause the state to oscillate!")

---

## Example 2: Quantum Superposition Experiments

What happens when we start with a superposition of classical patterns?

In [ ]:
print("Creating superposition of Block and Checkerboard patterns...\n")

# Create superposition state
block_state = create_pattern_state(patterns['Block (Still Life)'], (2, 2))
checker_state = create_pattern_state(patterns['Checkerboard'], (2, 2))

superposition_state = (block_state + checker_state) / np.sqrt(2)

print("Initial state: |ψ⟩ = (|Block⟩ + |Checkerboard⟩) / √2")
print("           = (|1111⟩ + |1001⟩) / √2\n")

# Evolve
time = 2.0
probs = qgol.evolve(H, time, 20, superposition_state)

# Analyze
print(f"After evolution (t={time}):")
print(f"\n{'State':<10} {'Probability':<15} {'Bar'}")
print("-" * 50)

top_states = np.argsort(probs)[-8:][::-1]
for idx in top_states:
    if probs[idx] > 0.01:
        binary = format(idx, '04b')
        bar = '█' * int(probs[idx] * 50)
        print(f"|{binary}⟩  {probs[idx]:>12.4f}  {bar}")

print("\n✨ Quantum interference creates new patterns that wouldn't")
print("   appear in classical GoL from these initial conditions!")

---

## Example 3: Parameter Sensitivity Analysis

How sensitive is the QGoL to evolution parameters?

In [ ]:
print("Analyzing sensitivity to Trotter steps...\n")

# Test different numbers of Trotter steps
trotter_steps_list = [5, 10, 20, 40, 80]
time = 1.0

initial = np.zeros(2**qgol.n_qubits)
initial[0] = 0.6
initial[5] = 0.8
initial = initial / np.linalg.norm(initial)

results = []
for n_steps in trotter_steps_list:
    probs = qgol.evolve(H, time, n_steps, initial)
    results.append(probs)

# Compare differences
print(f"{'Trotter Steps':<15} {'Max Prob Diff from n=80':<25}")
print("-" * 45)

reference = results[-1]  # n=80 as reference
for n_steps, probs in zip(trotter_steps_list, results):
    max_diff = np.max(np.abs(probs - reference))
    print(f"{n_steps:<15} {max_diff:<25.6f}")

# Visualize
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
for n_steps, probs in zip(trotter_steps_list, results):
    plt.plot(probs[:10], 'o-', label=f'n={n_steps}', alpha=0.7)
plt.xlabel('State Index', fontsize=11)
plt.ylabel('Probability', fontsize=11)
plt.title('Probability Distribution (first 10 states)', fontsize=12)
plt.legend(fontsize=9)
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
diffs = [np.max(np.abs(r - reference)) for r in results[:-1]]
plt.semilogy(trotter_steps_list[:-1], diffs, 'ro-', linewidth=2, markersize=8)
plt.xlabel('Number of Trotter Steps', fontsize=11)
plt.ylabel('Max Probability Difference', fontsize=11)
plt.title('Convergence with Trotter Steps', fontsize=12)
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n✓ More Trotter steps → better accuracy but longer computation time")

---

## Example 4: Entropy Evolution

Let's track how quantum entanglement evolves over time using von Neumann entropy.

In [ ]:
def calculate_entropy(probs):
    """Calculate von Neumann entropy from probability distribution."""
    # Remove zeros to avoid log(0)
    probs_nonzero = probs[probs > 1e-10]
    return -np.sum(probs_nonzero * np.log2(probs_nonzero))

print("Tracking entropy evolution...\n")

# Different initial states
pure_state = np.zeros(2**qgol.n_qubits)
pure_state[0] = 1.0

mixed_state = np.ones(2**qgol.n_qubits) / np.sqrt(2**qgol.n_qubits)

times = np.linspace(0, 5, 30)
entropy_pure = []
entropy_mixed = []

for t in times:
    probs_pure = qgol.evolve(H, t, 20, pure_state)
    probs_mixed = qgol.evolve(H, t, 20, mixed_state)
    
    entropy_pure.append(calculate_entropy(probs_pure))
    entropy_mixed.append(calculate_entropy(probs_mixed))

# Plot
plt.figure(figsize=(10, 5))
plt.plot(times, entropy_pure, 'b-', linewidth=2, label='Pure initial state |0000⟩')
plt.plot(times, entropy_mixed, 'r-', linewidth=2, label='Maximally mixed initial state')
plt.axhline(y=qgol.n_qubits, color='g', linestyle='--', linewidth=1.5, 
            label=f'Maximum entropy ({qgol.n_qubits} bits)')
plt.xlabel('Time', fontsize=12)
plt.ylabel('Von Neumann Entropy (bits)', fontsize=12)
plt.title('Entropy Evolution Under QGoL Hamiltonian', fontsize=13)
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"Initial entropy (pure state): {entropy_pure[0]:.4f} bits")
print(f"Final entropy (pure state): {entropy_pure[-1]:.4f} bits")
print(f"\nEntropy increase indicates spreading of quantum information")
print(f"and build-up of entanglement between cells!")

---

## Example 5: Animation of State Evolution

Let's create an animation showing how the grid probabilities evolve.

In [ ]:
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

def get_grid_probabilities(probs, grid_size):
    """Convert probability vector to grid of marginal probabilities."""
    rows, cols = grid_size
    n_qubits = rows * cols
    marginal = np.zeros(n_qubits)
    
    for state_idx, prob in enumerate(probs):
        binary = format(state_idx, f'0{n_qubits}b')
        for qubit in range(n_qubits):
            if binary[qubit] == '1':
                marginal[qubit] += prob
    
    return marginal.reshape(rows, cols)

print("Generating animation data...\n")

# Simulate trajectory
initial = np.zeros(2**qgol.n_qubits)
initial[5] = 1.0  # Start with |0101⟩

times = np.linspace(0, 4, 40)
grids = []

for t in times:
    probs = qgol.evolve(H, t, 20, initial)
    grid = get_grid_probabilities(probs, (qgol.rows, qgol.cols))
    grids.append(grid)

# Create animation
fig, ax = plt.subplots(figsize=(6, 5))

def animate(frame):
    ax.clear()
    im = ax.imshow(grids[frame], cmap='RdYlGn', vmin=0, vmax=1, interpolation='nearest')
    ax.set_title(f'QGoL Evolution (t={times[frame]:.2f})', fontsize=13)
    ax.set_xlabel('Column')
    ax.set_ylabel('Row')
    
    for i in range(qgol.rows):
        for j in range(qgol.cols):
            ax.text(j, i, f'{grids[frame][i, j]:.2f}', 
                   ha='center', va='center', color='black', fontsize=11)
    
    return [im]

anim = FuncAnimation(fig, animate, frames=len(times), interval=100, blit=True)
plt.close()  # Prevent static display

# Display animation
HTML(anim.to_jshtml())

# Note: In Jupyter, this will show an interactive animation
# You can also save it: anim.save('qgol_evolution.gif', writer='pillow')

---

## Exercise Section

Now it's your turn! Try these exercises to deepen your understanding.

### Exercise 1: Create Your Own Pattern

Design a custom 2×2 pattern and see how it evolves!

In [ ]:
# TODO: Define your pattern here
my_pattern = np.array([
    [0, 0],  # Modify these values (0 or 1)
    [0, 0]
])

# Create state and evolve
my_state = create_pattern_state(my_pattern, (2, 2))
final_probs = qgol.evolve(H, 2.0, 20, my_state)

# Analyze results
print("Your pattern:")
print(my_pattern)
print("\nTop 5 final states:")
for idx in np.argsort(final_probs)[-5:][::-1]:
    print(f"|{format(idx, '04b')}⟩: {final_probs[idx]:.4f}")

### Exercise 2: Compare Evolution Times

How does the final state depend on evolution time?

In [ ]:
# TODO: Test different evolution times
test_times = [0.5, 1.0, 2.0, 4.0]  # You can modify these

initial = np.zeros(2**qgol.n_qubits)
initial[0] = 1.0

# Your code here:
# 1. Evolve for each time
# 2. Calculate mean liveness for each
# 3. Plot the results

# Solution template:
mean_liveness_values = []
for t in test_times:
    # TODO: Implement evolution and analysis
    pass

### Exercise 3: Boundary Conditions Comparison

Compare periodic vs fixed boundary conditions.

In [ ]:
# TODO: Create two QGoL instances with different boundaries
qgol_periodic = QuantumGameOfLife(grid_size=(3, 3), periodic=True)
qgol_fixed = QuantumGameOfLife(grid_size=(3, 3), periodic=False)

# Build Hamiltonians
H_periodic = qgol_periodic.build_hamiltonian()
H_fixed = qgol_fixed.build_hamiltonian()

# TODO: 
# 1. Compare the number of terms
# 2. Evolve the same initial state under both
# 3. Compare the results

print(f"Periodic boundaries: {len(H_periodic.ops)} terms")
print(f"Fixed boundaries: {len(H_fixed.ops)} terms")
print(f"\nDifference: {len(H_periodic.ops) - len(H_fixed.ops)} terms")

# Your analysis code here...

---

## Challenge Problems

For advanced users!

### Challenge 1: Modified QGoL Rules

Modify the Hamiltonian to use different neighbor counts (e.g., birth at 2 neighbors instead of 3).

In [ ]:
# Hint: Modify the build_hamiltonian method to use different k values
# Standard: k = [2, 3]
# Your modification: k = [?, ?]

# Your code here...

### Challenge 2: Measure Entanglement

Calculate the entanglement entropy between two halves of the grid.

In [ ]:
# Hint: Use PennyLane's reduced density matrix functions
# qml.density_matrix(wires=[0, 1])

# Your code here...

### Challenge 3: Quantum Speedup?

Can you find a computational task where QGoL might offer an advantage over classical GoL?

In [ ]:
# This is an open-ended research question!
# Some ideas:
# - Pattern recognition using quantum interference
# - Exploring the full state space via superposition
# - Using entanglement for correlation detection

# Your ideas and code here...

---

## Summary

You've now explored:

✅ Classical GoL patterns in quantum contexts  
✅ Quantum superposition effects  
✅ Parameter sensitivity  
✅ Entropy and entanglement evolution  
✅ Animation and visualization  

**Next Steps:**
- Complete the exercises
- Try the challenge problems
- Explore your own research questions
- Share your findings!

---

**Happy quantum computing! 🎮🚀**